In [1]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [2]:
filenames=[]
for file in os.listdir('t20s'):
    filenames.append(os.path.join('t20s',file))

In [3]:
filenames[0:5]

['t20s/1388217.yaml',
 't20s/1388218.yaml',
 't20s/1388219.yaml',
 't20s/1388220.yaml',
 't20s/1388223.yaml']

In [ ]:
final_df=pd.DataFrame()
c=1
for file in tqdm(filenames):
    with open(file,'r') as f:
        data = safe_load(f)
        if data is not None:
            if isinstance(data, dict):
                data = [data]
            df = pd.json_normalize(data)
            df['match_id'] = c
            final_df = pd.concat([final_df, df], ignore_index=True)
        c+=1
final_df

<ipython-input-4-d18079cff6d7>:3: TqdmMonitorWarning: tqdm:disabling monitor support (monitor_interval = 0) due to:
can't start new thread
  for file in tqdm(filenames):
 66%|██████▋   | 1417/2135 [03:26<13:17,  1.11s/it]

In [ ]:
backup=final_df.copy()

In [ ]:
final_df

In [ ]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.supersubs.South Africa',
    'info.subersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

In [ ]:
final_df['info.gender'].value_counts()

In [ ]:
final_df=final_df[final_df['info.gender']=='male']
final_df.drop(columns=['info.gender'],inplace=True)
final_df

In [ ]:
final_df['info.match_type'].value_counts()

In [ ]:
final_df['info.overs'].value_counts()

In [ ]:
final_df=final_df[final_df['info.overs']==20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)
final_df

In [ ]:
import pickle
pickle.dump(final_df,open('dataset_l1.pkl','wb'))

In [ ]:
matches=pickle.load(open('dataset_l1.pkl','rb'))
matches

In [ ]:
count = 1
delivery_df = pd.DataFrame()
all_data = []

for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count += 1
        continue
    
    if 'innings' not in row or not row['innings'] or '1st innings' not in row['innings'][0]:
        count += 1
        continue
    
    if not row['innings'][0]['1st innings'].get('deliveries', []):
        count += 1
        continue
    
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings'].get('team', 'Unknown'))
            teams.append(row.get('info.teams', []))
            ball_of_match.append(key)
            batsman.append(ball[key].get('batsman', 'Unknown'))
            bowler.append(ball[key].get('bowler', 'Unknown'))
            
            runs_total = 0
            if 'runs' in ball[key] and 'total' in ball[key]['runs']:
                runs_total = ball[key]['runs']['total']
            runs.append(runs_total)
            
            city.append(row.get('info.city', 'Unknown'))
            venue.append(row.get('info.venue', 'Unknown'))
            
            if 'wicket' in ball[key] and 'player_out' in ball[key]['wicket']:
                player_dismissed.append(ball[key]['wicket']['player_out'])
            else:
                player_dismissed.append('0')
    
    loop_df = pd.DataFrame({
        'match_id': match_id,
        'teams': teams,
        'batting_team': batting_team,
        'ball': ball_of_match,
        'batsman': batsman,
        'bowler': bowler,
        'runs': runs,
        'player_dismissed': player_dismissed,
        'city': city,
        'venue': venue
    })
    all_data.append(loop_df)
    count += 1

delivery_df = pd.concat(all_data, ignore_index=True)

In [ ]:
delivery_df

In [ ]:
def bowl(row):
    for team in row['teams']:
        if team!=row['batting_team']:
            return team

In [ ]:
delivery_df['bowling_team']=delivery_df.apply(bowl,axis=1)

In [ ]:
delivery_df

In [ ]:
delivery_df.drop(columns=['teams'],inplace=True)

In [ ]:
delivery_df

In [ ]:
teams=[
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'
]

In [ ]:
delivery_df=delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df=delivery_df[delivery_df['bowling_team'].isin(teams)]

In [ ]:
delivery_df

In [ ]:
output=delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [ ]:
output

In [ ]:
pickle.dump(output,open('dataset_l2.pkl','wb'))

In [ ]:
df.isnull().sum()

In [ ]:
df[df['city'].isnull()]['venue'].value_counts()